In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import json
import re
import html
import spacy
import spacy_fastlang

In [2]:
# dfs = [pd.read_csv('text_batch1.csv', encoding="utf-8", lineterminator='\n')]
# for i in range(2, 5):
#     single_df_str = 'text_batch%d.csv' % i
#     print(i)
#     dfs.append(pd.read_csv(single_df_str, encoding="utf-8", lineterminator='\n'))
# df = pd.concat(dfs, ignore_index=True)
# df.to_csv('allforms.csv', encoding='utf-8', index=False)
# df.describe()

In [3]:
df = pd.read_csv('allforms.csv', encoding="utf-8", lineterminator='\n')

In [4]:
print(df.isna().sum())
df.dropna(inplace=True)
df.drop_duplicates()

id           0
sentence    47
dtype: int64


,id,sentence
0,12502728116,About You.... SEND RESERVATION. Full Name. E-m...
1,21503493135952,Payment Form. CLICK TO PROCESS YOUR PAYMENT . ...
2,50556096982971,Submit. Company Name. Contact Name. E-mail. De...
3,60345507222952,Full Name. Submit. E-mail. Phone Number. Reque...
4,60405483339961,Lost and/or Found Form . Submit. Fu...
...,...,...
399590,81425390115349,Envoyer. E-mail. Téléphone. Type de demande. <...
399591,81426382540151,"Submit. Job Safety Analysis - Shop, Yard & Tru..."
399592,81428439543159,SCBA Check. Submit. Vehicle Number - Brush. Cr...
399593,81431483343857,Enhanced Due Diligence Form . Transaction Date...


In [5]:
html_regex = re.compile('<\/?[^>]*>')
url_regex = re.compile('\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*')

In [6]:
def clean_text(x):
    global html_regex, url_regex
    x[1] = re.sub(html_regex, '', x[1])
    x[1] = re.sub(url_regex, '', x[1])
    x[1] = ' '.join(x[1].split())
    x[1] = html.unescape(x[1])
    x[1] = ''.join(word for word in x[1] if word.isprintable())
    return x

In [7]:
def determine_acceptance(x):
    if not x[1]:
        return False
    elif not x[1].isascii():
        return False
    return True

In [8]:
df_list = df.values.tolist()
new_df_list = [clean_text(x) for x in df_list if determine_acceptance(x)]
df = pd.DataFrame(new_df_list, columns =['id', 'sentence'])
len(new_df_list)

212107

In [9]:
nlp = spacy.load('en_core_web_md')
nlp.add_pipe('language_detector')

In [10]:
scores = {}
blacklist = []
tcount = df.shape[0]
for row in tqdm(df.itertuples(index=False), total=tcount):
    if len(row.sentence) < 2:
        blacklist.append(row.id)
        continue
    doc = nlp(row.sentence)
    if 'en' == doc._.language:
        scores[row.id] = doc._.language_score
    else:
        scores[row.id] = 0

100%|██████████| 212107/212107 [1:03:15<00:00, 55.89it/s] 


In [11]:
print(len(blacklist), blacklist)

6 [211516749597064, 211084488493362, 210944679287066, 210758136460050, 202445800369050, 211442140693044]


In [12]:
df = df[~df['id'].isin(blacklist)]

In [13]:
df['eng_score'] = df['id'].map(scores)

In [14]:
df.head()

,id,sentence,eng_score
0,12502728116,About You.... SEND RESERVATION. Full Name. E-m...,0.794774
1,50556096982971,Submit. Company Name. Contact Name. E-mail. De...,0.618418
2,60345507222952,Full Name. Submit. E-mail. Phone Number. Reque...,0.811168
3,60405483339961,Lost and/or Found Form . Submit. Full Name. To...,0.910910
4,72682483684166,Contact Us. Submit. Name. E-mail. How Can We H...,0.624258


In [15]:
df.to_csv('eng_score_new.csv', encoding='utf-8', index=False)